In [ ]:
# download dependencies

In [2]:
! pip install PyMuPDF # dependencies for reading PDF files

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [7]:
! pip install --upgrade pymupdf

   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
    --------------------------------------- 0.3/16.2 MB ? eta -:--:--
   - -------------------------------------- 0.8/16.2 MB 2.1 MB/s eta 0:00:08
   --- ------------------------------------ 1.3/16.2 MB 2.3 MB/s eta 0:00:07
   ---- ----------------------------------- 1.8/16.2 MB 2.4 MB/s eta 0:00:07
   ----- ---------------------------------- 2.1/16.2 MB 2.5 MB/s eta 0:00:06
   ----- ---------------------------------- 2.1/16.2 MB 2.5 MB/s eta 0:00:06
   ----- ---------------------------------- 2.1/16.2 MB 2.5 MB/s eta 0:00:06
   ---------- ----------------------------- 4.2/16.2 MB 2.7 MB/s eta 0:00:05
   ---------- ----------------------------- 4.5/16.2 MB 2.4 MB/s eta 0:00:05
   ------------- -------------------------- 5.5/16.2 MB 2.6 MB/s eta 0:00:05
   --------------- --------

In [8]:
import os
import pymupdf   # type: ignore

In [9]:
def load_pdfs_from_folder(folder_path):
    pdf_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.pdf')]
    return pdf_files

def extract_text_from_pdf(pdf_path):
    doc = pymupdf.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

In [12]:
doc_path = "../Data/pdfs_for_Vanilla_RAG"

In [13]:
loaded_files = load_pdfs_from_folder(doc_path)
extracted_first_pdf = extract_text_from_pdf(loaded_files[0])

In [10]:
# Naive chunking
def chunk_text_into_paragraphs(text):
    paragraphs = []
    current_paragraph = []

    for line in text.splitlines():
        line = line.strip()
        if line:
            current_paragraph.append(line)
        else:
            if current_paragraph:
                paragraphs.append(" ".join(current_paragraph))
                current_paragraph = []
    if current_paragraph:
        paragraphs.append(" ".join(current_paragraph))
    
    return paragraphs

def process_pdfs_in_folder(folder_path):
    pdf_files = load_pdfs_from_folder(folder_path)
    all_paragraphs = []

    for pdf_file in pdf_files:
        text = extract_text_from_pdf(pdf_file)
        paragraphs = chunk_text_into_paragraphs(text)
        
        for i, paragraph in enumerate(paragraphs):
            all_paragraphs.append({
                'paragraph': i + 1,
                'content': paragraph,
                'source': pdf_file
            })
    
    return all_paragraphs

In [16]:
folder_path = "../Data/pdfs_for_Vanilla_RAG"
chunked_pdfs = process_pdfs_in_folder(folder_path)

In [17]:
chunked_pdfs[0]

{'paragraph': 1,
 'content': 'Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need Ashish Vaswani∗ Google Brain avaswani@google.com Noam Shazeer∗ Google Brain noam@google.com Niki Parmar∗ Google Research nikip@google.com Jakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Research llion@google.com Aidan N. Gomez∗† University of Toronto aidan@cs.toronto.edu Łukasz Kaiser∗ Google Brain lukaszkaiser@google.com Illia Polosukhin∗‡ illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence an

# Using a transformer

In [11]:
! pip install -U sentence-transformers

  Using cached tqdm-4.67.0-py3-none-any.whl.metadata (57 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached tokenizers-0.20.3-cp312-none-win_amd64.whl.metadata (6.9 kB)
  Using cached charset_normalizer-3.4.0-cp312-cp312-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
Using cached huggingface_hub-0.26.2-py3-none-any.whl (447 kB)
   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/203.0 MB

In [14]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)

c:\Users\Lisara\anaconda3\envs\research_meth_env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Lisara\anaconda3\envs\research_meth_env\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lisara\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate

[[-2.02629730e-04  8.14801455e-02  3.13617848e-02  2.92064785e-03
   2.61564497e-02  2.90740170e-02  7.82618895e-02 -1.80412480e-03
   1.01344302e-01 -4.51712050e-02  5.84350117e-02 -1.53200431e-02
   5.49956523e-02 -9.86434743e-02 -3.50252502e-02  8.45675077e-03
   1.58609524e-02  1.05626974e-02 -3.42710018e-02 -4.75063873e-03
   9.99020785e-02 -2.06018146e-02 -4.47837003e-02  3.12135927e-02
  -1.19241131e-02 -5.15015572e-02 -1.33605916e-02  1.89621523e-02
   9.76810232e-02 -5.44110537e-02 -3.43314223e-02  8.12905505e-02
   4.88119163e-02 -1.10283978e-02  2.13518925e-02  1.27189932e-02
  -1.43967392e-02  3.62864323e-02 -7.61232823e-02  3.23294066e-02
   2.08102409e-02 -4.22016531e-02  9.12906677e-02  2.08529476e-02
  -3.08017209e-02 -8.38504806e-02  1.30890822e-02 -3.00631076e-02
   4.11229171e-02 -1.27495363e-01 -7.78027549e-02 -3.93412672e-02
   1.52603071e-03 -2.80108396e-02  3.41662355e-02  1.46712149e-02
  -7.71653652e-02  1.63619608e-01  4.11294885e-02 -5.24460115e-02
  -4.18772

In [ ]:
embeddings = model.encode([chunked_pdf['content'] for chunked_pdf in chunked_pdfs])

In [18]:
# Vectorization

In [19]:
embeddings = model.encode([chunked_pdf['content'] for chunked_pdf in chunked_pdfs])

In [20]:
# merge chunked data with vector data of them
points = [
    {
        "id": i,
        "vector": embedding,
        "payload": {
            "content": doc['content'],
            "paragraph": doc['paragraph'],
            "source": doc['source']
        },
    }
    for i, (embedding, doc) in enumerate(zip(embeddings, chunked_pdfs))
]

# Vector DB and storing

In [23]:
! pip install chromadb

  Using cached chromadb-0.5.18-py3-none-any.whl.metadata (6.8 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.115.4-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.32.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached posthog-3.7.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached onnxruntime-1.20.0-cp312-cp312-win_amd64.whl.metadata (4.6 kB)
  Using cached opentelemetry_api-1.28.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.28.0-py3-none

  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


In [ ]:
! pip install --upgrade chromadb

In [27]:
import chromadb
chroma_client = chromadb.Client()

ModuleNotFoundError: No module named 'chromadb'

# Vector storing